In [1]:
!pip install umap-learn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
from sentence_transformers import SentenceTransformer
import umap 

import pandas as pd 
import re

In [6]:
df = pd.read_csv("archive/TheOffice_transcripts.csv", on_bad_lines= 'skip')

df.columns = ["sentence"]

In [7]:
for i in range(len(df)):

    string = str(df["sentence"][i])
    seperator = ';'
    df["sentence"][i] = string.split(seperator, 1)[0]

    string2 = str(df["sentence"][i])
    seperator = ':'
    

In [8]:
df

,sentence
0,Michael: All right Jim. Your quarterlies look ...
1,Pam: Well. I don't know.
2,Pam: What?
3,Michael: Any messages?
4,Pam: You haven't told me.
...,...
34028,I'm going to miss this incredible family here ...
34029,This has been my other family for nine years. ...
34030,And it's a tough thing.
34031,It's just been everything to me.
